In [ ]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as LA

**SYSTEMS FUNCTIONS**

notes:
- for LDS: can try inital hidden state 0.
- for NAR: can try to change p, can try to bound initial states, as well as matrices A.
- for RNN: can see how we define the weight matrix.

In [23]:
def system_lds(T, d_hid, d_in, d_out=1, seed=0):
    """
    Simulates a Linear Dynamical System (LDS) according to the equations:
        x_{t+1} = A x_t + B u_t
        y_t     = C x_t
    
    ARGS:
        T      : int       (number of time steps)
        d_hid  : int       (hidden state dimension)
        d_in   : int       (input dimension)
        d_out  : int       (output dimension)
        seed   : int       (random seed for reproducibility)

    Returns:
        U ∈ ℝ^{T×d_in}  (inputs)
        Y ∈ ℝ^{T×d_out} (observed output)
    """
    np.random.seed(seed)

    # Real diagonalizable A with eigenvalues in (-1, 1)
    Q, _ = np.linalg.qr(np.random.randn(d_hid, d_hid))
    eigvals = np.random.uniform(-1, 1, size=d_hid)
    A = Q @ np.diag(eigvals) @ Q.T

    # Random system matrices
    B = np.random.randn(d_hid, d_in)
    C = np.random.randn(d_out, d_hid)

    # Initial state and storage
    x = np.random.uniform(-1, 1, size=(d_hid, 1)) # random initial hidden state
    U = np.zeros((T, d_in))     # inputs matrix
    Y = np.zeros((T, d_out))    # observed output matrix

    # Simulate
    for t in range(T):
        u = np.random.uniform(-1, 1, size=(d_in, 1)) # random input at each step
        U[t] = u.flatten()             # store input
        y = C @ x                      # compute output
        Y[t] = y.flatten()             # store output
        x = A @ x + B @ u              # state update

    return U, Y

In [34]:
def system_nar(T, d_out=1, p=2, seed=0):
    """
    Simulates a Nonlinear Autoregressive (NAR) system according to the equation:
        x_t = tanh(sum_k A_k @ x_{t-k})

    ARGS:
        T      : int       (number of time steps)
        d_out  : int       (output dimension)
        p      : int       (order of the autoregressive model)
        seed   : int       (random seed for reproducibility)
    
    Returns:
        Y ∈ ℝ^{T×d_out}  (observed output)
    """

    np.random.seed(seed)

    A_k = [0.8 * np.random.randn(d_out, d_out)/np.sqrt(d_out) for _ in range(p)] # p - A (dxd) matrices   
    X = np.zeros((T, d_out))        # observed output matrix

    # random initial states
    for i in range(p):
        X[i] = np.random.randn(d_out)

    for t in range(p, T):
      past_states = [X[t-k] for k in range(1, p+1)] # collect past p states
      x_t = np.zeros(d_out)
      for k in range(1, p+1):
          x_t += A_k[k-1] @ X[t-k]
      X[t] = np.tanh(x_t)
      
    return X

In [35]:
def system_logistic(T, d_out=1, r=2.0, seed=0):
    """
    Simulate a Logistic map system according to the equation:
        x[t+1] = r * x[t] * (1 - x[t])
    (nonlinear, chaotic for r>3.57)

    ARGS:
        T      : int       (number of time steps)
        d_out  : int       (output dimension)
        r      : float     (logistic map parameter)
        seed   : int       (random seed for reproducibility)

    Returns:
        X ∈ ℝ^{T×d_out}  (observed output)
    """

    np.random.seed(seed)

    X = np.zeros((T, d_out))
    x0 = np.random.rand(d_out)

    X[0] = x0
    for t in range(T-1):
        X[t+1] = r*X[t]*(1.0 - X[t])

    return X

In [36]:
def system_lorenz(T, sigma=10.0, beta=8.0/3.0, rho=28.0, dt=0.01, seed=0):
    """
    Simulates the Lorenz system according to the equations:
        dx/dt = sigma * (y - x)
        dy/dt = x * (rho - z) - y
        dz/dt = x * y - beta * z

    ARGS:
        T      : int       (number of time steps)
        sigma  : float     (Lorenz parameter)
        beta   : float     (Lorenz parameter)
        rho    : float     (Lorenz parameter)
        dt     : float     (time step size)
        seed   : int       (random seed for reproducibility)

    Returns:
        X ∈ ℝ^{T×3}  (observed output)
    """
    np.random.seed(seed)

    # Lorenz dynamics
    def f(x):
        return np.array([
            sigma*(x[1] - x[0]),
            x[0]*(rho - x[2]) - x[1],
            x[0]*x[1] - beta*x[2]
        ])

    # Simulate true 3D system
    X  = np.zeros((T, 3))

    x = x0 = np.array([1.0, 1.0, 1.0])
    X[0] = x0
    for t in range(T):
        k1 = f(x)
        k2 = f(x + 0.5*dt*k1)
        k3 = f(x + 0.5*dt*k2)
        k4 = f(x + dt*k3)
        x = x + (dt/6.0)*(k1 + 2*k2 + 2*k3 + k4)
        X[t] = x

In [37]:
def system_rnn(T, d_out=1, rho_h=0.95, seed=0):
    """
     Simulates a Recurrent Neural Network (RNN) system according to the equation:
        x_{t+1} = tanh(Wx * x_t)

     ARGS:
        T      : int       (number of time steps)
        d_out  : int       (output dimension)
        rho_h  : float     (desired spectral radius of the weight matrix)
        seed   : int       (random seed for reproducibility)
    
    Returns:
        X ∈ ℝ^{T×d_out}  (observed output)
    """
    np.random.seed(seed)

    W = np.random.randn(d_out, d_out) / np.sqrt(d_out)    # weight matrix
    spec = np.linalg.svd(W, compute_uv=False)[0]
    W_h = (rho_h / spec) * W                  # Scale W so that its spectral norm ≈ rho_h
    X = np.zeros((T, d_out))
    X[0] = np.random.randn(d_out)
    for t in range(T - 1):
        X[t+1] = np.tanh(W_h @ X[t])
    return X

**BASES FUNCTIONS**

In [ ]:
def spectral_basis(U, Y):


In [ ]:
def chebyshev_dct_basis(T):
    """
    Chebyshev and DCT basis
    """
    n = np.arange(T)[:, None]     # column vector with vals 0 to T-1
    m = np.arange(T)[None, :]     # row vector with vals 0 to T-1
    C = np.cos(np.pi*(n + 0.5)*m/T)   # computes the C (TxT) matrix where each element: cos(pi*(n + 0.5)*k/T)
    C[:, 0] = C[:, 0] / np.sqrt(T)    # normalize the first col by 1/sqrt(T)
    if T > 1: C[:, 1:] = C[:, 1:] * np.sqrt(2.0/T)    # normalize remaining cols by sqrt(2/T)
    Q, _ = np.linalg.qr(C)                            # orthonormal basis Q spanning the same subspace as C (TxT)
    return Q[:, :k_i]                                 # return the first k dim of the filter


In [ ]:
def fft_lowfreq_basis(T, k, u):
    """
    Real Fourier basis (cos/sin) up to frequency k (inclusive).
    Returns orthonormal basis vectors (T x m).
    """
    cols = [np.ones(T) / np.sqrt(T)]            # normalized DC component
    for j in range(1, min(k, T//2) + 1):        # stops at k or Nyquist whatever is smaller
        cols.append(np.sqrt(2/T) * np.cos(2*np.pi*j*t/T))     # consine part
        if j < T/2:                                           # no sine at Nyquist freq when T is even
            cols.append(np.sqrt(2/T) * np.sin(2*np.pi*j*t/T)) # sine part

    M = np.stack(cols, axis=1)
    return M

In [ ]:
def time_pca_basis(D, k=5):

**EXPERIMENTS & PLOT GENERATION**

In [ ]:
# Set variables
T, dhid, din = 1024, 10, 10

# Generate synthetic systems data
U_lds, Y_lds = system_lds(T, dhid, din)
X_nar = system_nar(T)
X_logistic = system_logistic(T)
X_lorenz = system_lorenz(T)
X_rnn = system_rnn(T)

# Project to all bases

# Plot results
# Plot A: Raw signal

# Plot B: Frobenius error over k